In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

import numpy as np

In [21]:
excel_file_path = 'Data/prc_hicp_midx__custom_8953455_spreadsheet.xlsx'
sheet_name = 'Sheet 1'
skip_rows = 8  # Skip the first 8 rows (start from the 9th row)

# Read the specific sheet from the Excel file, skipping the specified rows
data = pd.read_excel(excel_file_path, sheet_name=sheet_name, skiprows=skip_rows)

# Filter and keep only columns that do not start with "Unnamed"
data = df.loc[:, ~df.columns.str.startswith('Unnamed')]

/opt/mamba/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
data

,TIME,1996-01,1996-02,1996-03,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,...,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11
0,GEO (Labels),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"European Union (EU6-1958, EU9-1973, EU10-1981,...",69.4,69.7,70,70.19,70.37,70.39,70.28,70.32,70.53,...,124.95,126.05,126.83,126.88,127.22,127.17,127.78,128.11,128.27,:
2,European Union - 27 countries (from 2020),:,:,:,:,:,:,:,:,:,...,124.43,125.52,126.3,126.35,126.69,126.64,127.24,127.57,127.73,:
3,European Union - 28 countries (2013-2020),:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
4,"Euro area (EA11-1999, EA12-2001, EA13-2007, EA...",70.97,71.29,71.54,71.66,71.83,71.84,71.84,71.8,71.93,...,121.24,122.34,123.12,123.15,123.47,123.36,124.03,124.43,124.54,123.86
5,Euro area – 20 countries (from 2023),:,:,:,:,:,:,:,:,:,...,121.25,122.35,123.13,123.17,123.48,123.37,124.04,124.44,124.55,:
6,Euro area - 19 countries (2015-2022),70.4,70.71,71.01,71.14,71.32,71.33,71.31,71.27,71.44,...,121.24,122.34,123.11,123.14,123.45,123.32,124,124.4,124.51,:
7,Belgium,69.91,69.98,70.12,70.47,70.61,70.47,70.4,70.4,70.54,...,125.99,126.23,124.65,124.91,124.79,124.05,127.2,126.73,127.15,127.12
8,Bulgaria,:,:,:,:,:,:,:,:,:,...,132.12,132.9,133.63,133.3,133.45,134.99,135.61,135.18,135.44,:
9,Czechia,57,57.3,57.6,58.1,58.4,58.9,59.4,59.5,59.6,...,147.3,147.7,147.5,147.9,148.4,149,149.3,148.1,148.1,:


In [22]:
df = data.iloc[1:-6]

In [20]:
df

,TIME,1996-01,1996-02,1996-03,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,...,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11
1,"European Union (EU6-1958, EU9-1973, EU10-1981,...",69.4,69.7,70,70.19,70.37,70.39,70.28,70.32,70.53,...,124.95,126.05,126.83,126.88,127.22,127.17,127.78,128.11,128.27,:
2,European Union - 27 countries (from 2020),:,:,:,:,:,:,:,:,:,...,124.43,125.52,126.3,126.35,126.69,126.64,127.24,127.57,127.73,:
3,European Union - 28 countries (2013-2020),:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
4,"Euro area (EA11-1999, EA12-2001, EA13-2007, EA...",70.97,71.29,71.54,71.66,71.83,71.84,71.84,71.8,71.93,...,121.24,122.34,123.12,123.15,123.47,123.36,124.03,124.43,124.54,123.86
5,Euro area – 20 countries (from 2023),:,:,:,:,:,:,:,:,:,...,121.25,122.35,123.13,123.17,123.48,123.37,124.04,124.44,124.55,:
6,Euro area - 19 countries (2015-2022),70.4,70.71,71.01,71.14,71.32,71.33,71.31,71.27,71.44,...,121.24,122.34,123.11,123.14,123.45,123.32,124,124.4,124.51,:
7,Belgium,69.91,69.98,70.12,70.47,70.61,70.47,70.4,70.4,70.54,...,125.99,126.23,124.65,124.91,124.79,124.05,127.2,126.73,127.15,127.12
8,Bulgaria,:,:,:,:,:,:,:,:,:,...,132.12,132.9,133.63,133.3,133.45,134.99,135.61,135.18,135.44,:
9,Czechia,57,57.3,57.6,58.1,58.4,58.9,59.4,59.5,59.6,...,147.3,147.7,147.5,147.9,148.4,149,149.3,148.1,148.1,:
10,Denmark,70.4,70.8,71.3,71.5,71.7,71.6,71.5,71.6,72,...,118,117.8,118.1,116.3,116.7,119.1,118.1,117.8,118.1,:


In [23]:
# Delete rows where the first column starts with "Euro"
df = df[~df.iloc[:, 0].str.startswith("Euro")]

In [26]:
# Replace ":" with NaN in the entire DataFrame
df.replace(":", np.nan, inplace=True)

In [27]:
df

,TIME,1996-01,1996-02,1996-03,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,...,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11
7,Belgium,69.91,69.98,70.12,70.47,70.61,70.47,70.40,70.40,70.54,...,125.99,126.23,124.65,124.91,124.79,124.05,127.20,126.73,127.15,127.12
8,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,132.12,132.90,133.63,133.30,133.45,134.99,135.61,135.18,135.44,NaN
9,Czechia,57.00,57.30,57.60,58.10,58.40,58.90,59.40,59.50,59.60,...,147.30,147.70,147.50,147.90,148.40,149.00,149.30,148.10,148.10,NaN
10,Denmark,70.40,70.80,71.30,71.50,71.70,71.60,71.50,71.60,72.00,...,118.00,117.80,118.10,116.30,116.70,119.10,118.10,117.80,118.10,NaN
11,Germany,75.10,75.60,75.70,75.60,75.70,75.80,76.00,75.90,75.90,...,123.80,125.10,125.80,125.60,126.10,126.70,127.20,127.40,127.20,NaN
12,Estonia,42.87,44.24,44.95,45.72,45.95,46.23,46.36,45.92,46.11,...,146.14,147.20,150.04,150.08,151.30,150.56,151.25,151.17,150.45,150.47
13,Ireland,68.10,68.60,68.90,68.80,68.90,69.10,68.90,69.30,69.70,...,115.80,116.90,117.20,117.50,118.40,118.60,119.20,119.30,119.50,118.20
14,Greece,59.04,58.92,60.64,61.25,61.74,61.86,60.76,60.70,62.11,...,112.51,114.28,115.55,116.17,117.23,116.08,115.87,118.04,118.17,116.91
15,Spain,63.95,64.08,64.34,64.73,64.99,64.93,64.99,65.19,65.38,...,117.82,119.06,119.61,119.44,120.13,120.03,120.68,121.38,121.69,121.01
16,France,74.13,74.42,74.90,74.99,75.19,75.11,74.94,74.78,75.02,...,118.55,119.76,120.55,120.46,120.71,120.77,122.10,121.40,121.61,121.27


In [34]:
shapefile_path = 'CNTR_BN_01M_2020_3035.shp'
gdf_countries = gpd.read_file(shapefile_path)

DriverError: Unable to open CNTR_BN_01M_2020_3035.shx or CNTR_BN_01M_2020_3035.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.